In [10]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', lambda x: '%.2f' % x)

#### нужно получить распредение населения по возрастным группам
#### 0-17, 18-60, 61 и старше в разрезе мужчин и женщин. Необходимо прогнозное
#### количество женщин детородного возраста, чтобы по регулируемому показателю 
#### фертильности, получить прогноз всего населения Украины.
#### Далее, зная фертильность и количество детей, можно спрогнозировать расходы
#### которые должно взять на себя государство, чтобы увеличилась рождаемость.

#### try to recover demographic model by AI

In [2]:
pop=pd.read_excel('data/pop_per_year.xlsx')

In [3]:
pop2013=pop[pop['рік']==2013]#исходный факт
pop2013.head(2)

,рік,вік,Обидві статі,жінки,чоловіки
81,2013,0,517301,250481,266820
82,2013,1,498371,241404,256967


In [4]:
pop2017=pop[pop['рік']==2017]#исходный факт
pop2017.head(2)

,рік,вік,Обидві статі,жінки,чоловіки
405,2017,0,394626,190833,203793
406,2017,1,408645,197600,211045


In [11]:
#коэффициенты відтворення та міграції по мужчинам и женщинам
#выделены из файлы модели, нужно сделать отдельными сериями,чтобы не привязываться к файлам
koef_vidtvorennya_nas_f=pd.read_excel('D:/Users/o.lukasevych/Documents/Python_Scripts/Ukr_budget/excel/Copy of Демографическая модель изменения населения Украины.xlsx',\
                                     sheet_name='Sheet1')['відтворення населення жінки']
koef_vidtvorennya_nas_m=pd.read_excel('D:/Users/o.lukasevych/Documents/Python_Scripts/Ukr_budget/excel/Copy of Демографическая модель изменения населения Украины.xlsx',\
                                     sheet_name='Sheet1')['відтворення населення чоловіка']
koef_mizhnar_saldo_f=pd.read_excel('D:/Users/o.lukasevych/Documents/Python_Scripts/Ukr_budget/excel/Copy of Демографическая модель изменения населения Украины.xlsx',\
                                     sheet_name='Sheet1')['міжнародне сально жінки']
koef_mizhnar_saldo_m=pd.read_excel('D:/Users/o.lukasevych/Documents/Python_Scripts/Ukr_budget/excel/Copy of Демографическая модель изменения населения Украины.xlsx',\
                                     sheet_name='Sheet1')['міжнародне сальдо чоловіка']

In [6]:
koef_zhyvonar=99.3835113134263 #коэффициент живорожденных от рожденных
koef_mertvonar=0.616488686573717 #коэффициент мертворожденных от родженных
koef_pomer=1.45894635287282 #коэффициент умерших от всего населения

koef_usogo_nar_f=48.3748938322502 #коэффициент всего рожденных женщин
koef_zhivonar_f=48.3876667210801 #коэффициент живорожденных женщин
koef_mertvonar_f=46.3157894736842 #коэффициент мертворожденных женщин
koef_pomer_f=50.9026011137936 #коэффициент умерших женщин

  #коэффициенты по мужчинам
koef_usogo_nar_m=51.6251061677498
koef_zhivonar_m=51.6123332789199
koef_mertvonar_m=53.6842105263158
koef_pomer_m=49.0973988862064

    #коэффициенты миграции
koef_migrant_f=30.6820386779639
koef_migrant_m=69.3179613220361

# на входе функции серия женщин по годам жизни,серия мужчины по годам жизни, всего населения
# кол-во детей на 1000 женщин,коеффициент миграции
def model(f_years,m_years,popTotal,koef_fert,koef_migrant):
    fertilnyh=f_years[18:50].sum()
    narodzhennyh=fertilnyh*koef_fert/1000
    zhivonarodzhennyh=narodzhennyh*koef_zhyvonar/100
    mertvonarodzhennyh=narodzhennyh*koef_mertvonar/100
    pomer=popTotal*koef_pomer/100
    
    all_data=pd.Series({'fertilnyh':fertilnyh,'narodzhennyh':narodzhennyh,\
    'zhivonarodzhennyh':zhivonarodzhennyh,\
    'mertvonarodzhennyh':mertvonarodzhennyh,\
    'pomer':pomer})
    
    #количества по женщинам за текущий год
    usogo_nar_f=narodzhennyh*koef_usogo_nar_f/100 #всего рожденных женщин
    zhivonar_f=koef_zhivonar_f/100*zhivonarodzhennyh #количество живорожденных женщин
    mertvonar_f=koef_mertvonar_f/100*mertvonarodzhennyh #количество мертворожденных женщин
    pomer_f=koef_pomer_f/100*pomer #количество умерших женщин
    
    data_f=pd.Series({'usogo_nar_f':usogo_nar_f,'zhivonar_f':zhivonar_f,\
                      'mertvonar_f':mertvonar_f,'pomer_f':pomer_f})
    
    #количества по мужчинам за текущий год
    zhivonar_m=koef_zhivonar_m/100*zhivonarodzhennyh
    usogo_nar_m=narodzhennyh*koef_usogo_nar_m/100
    mertvonar_m=koef_mertvonar_m/100*mertvonarodzhennyh
    pomer_m=koef_pomer_m/100*pomer
    
    data_m=pd.Series({'usogo_nar_m':usogo_nar_m,'zhivonar_m':zhivonar_m,\
                     'mertvonar_m':mertvonar_m,'pomer_m':pomer_m})
    
    #серии відтворення населення
    vidtvorennya_nas_f=pd.Series(pomer_f*koef_vidtvorennya_nas_f/100*(-1))
    vidtvorennya_nas_m=pd.Series(pomer_m*koef_vidtvorennya_nas_m/100*(-1))
    
    # серии мигрантов
    migrant=popTotal*koef_migrant/100
    migrant_f=migrant*koef_migrant_f/100
    migrant_f_years=pd.Series(koef_mizhnar_saldo_f*migrant_f/100)
    migrant_m=migrant*koef_migrant_m/100
    migrant_m_years=pd.Series(koef_mizhnar_saldo_m*migrant_m/100)
    
    # серии женщин следующего года по годам жизни
    rest1=f_years[:-1]
    rest2=vidtvorennya_nas_f.drop(0).reset_index(drop=True)
    rest3=migrant_f_years.drop(0).reset_index(drop=True)
    rest1=rest1.astype('float64')
    rest4=rest1+rest2+rest3
    rest4=rest4[:-1]
    rest_first=zhivonar_f+vidtvorennya_nas_f[0]+migrant_f_years[0]
    rest_last=pd.Series(f_years.iloc[-1]+f_years.iloc[-2]+\
                vidtvorennya_nas_f.iloc[-1]+\
                migrant_f_years.iloc[-1])
    rest=rest4.append(rest_last).reset_index(drop=True)
    rest.index=rest.index+1
    rest_f=pd.concat([rest, pd.Series(rest_first)])
    female_next=rest_f.sort_index()
    
    rest_series=pd.Series({'rest':rest,'rest1':rest1,\
                           'rest2':rest2,'rest3':rest3,\
                           'rest4':rest4,\
                           'rest_first':rest_first,'rest_last':rest_last})
    
    # серии мужчин следующего года по годам жизни
    mrest1=m_years[:-1]
    mrest2=vidtvorennya_nas_m.drop(0).reset_index(drop=True)
    mrest3=migrant_m_years.drop(0).reset_index(drop=True)
    mrest1=mrest1.astype('float64')
    mrest4=mrest1+mrest2+mrest3
    mrest4=mrest4[:-1]
    mrest_first=zhivonar_m+vidtvorennya_nas_m[0]+migrant_m_years[0]
    mrest_last=pd.Series(m_years.iloc[-1]+m_years.iloc[-2]+\
                vidtvorennya_nas_m.iloc[-1]+\
                migrant_m_years.iloc[-1])
    mrest=mrest4.append(mrest_last).reset_index(drop=True)
    mrest.index=mrest.index+1
    rest_m=pd.concat([mrest, pd.Series(mrest_first)])
    male_next=rest_m.sort_index()
    pop_next=female_next+male_next
    pop_next_total=pop_next.sum()
    
     
    
    return female_next,male_next,pop_next_total,vidtvorennya_nas_f       
   

In [7]:
def prognos(year,f_years,m_years,kfert):
    z=[]
    ye=[]
    pop_total=f_years.sum()+m_years.sum()
    for i in range(len(kfert)):
        fy=model(f_years,m_years,pop_total,kfert[i],0.136060581939936)[0]
        my=model(f_years,m_years,pop_total,kfert[i],0.136060581939936)[1]
        pt=model(f_years,m_years,pop_total,kfert[i],0.136060581939936)[2]
        z.append(model(f_years,m_years,pop_total,kfert[i],0.136060581939936)[2])
        f_years=fy
        m_years=my
        pop_total=pt
        ye.append(year+1+i)
        
    return pd.DataFrame(z,ye)

In [8]:
f=pop2017[' жінки'].reset_index(drop=True)
m=pop2017[' чоловіки'].reset_index(drop=True)

kfert_good=np.insert(np.repeat(95,40),[0],[46,60,65,70,75,80,85,90])
kfert_bad=np.repeat(46,37)

prognos(2017,f,m,np.repeat(46,100)).iloc[-1,:]

0   24692581
Name: 2117, dtype: float64